## The DSL backend

As mentioned in the introduction tutorial, backends are responsible for keeping track of any tensors created/removed during the creation or contraction of tensor networks. The Domain Specific Language (DSL) backend does this by writing tensor data, and commands describing the manipulation of it, to disk instead of saving the data in memory and acting on it interactively. We write the commands to disk using a domain specific language which is stored in plain text files suffixed with `.tl` default for tensor language.

With a DSL backend, when a tensor network is created with PicoQuant, all of the associated tensor data will be written to a HDF5 file. When we call functions to contract or manipulate tensors contained in a tensor network, instead of acting on the tensor data in the HDF5 file directly, the DSL backend will use a domain specfic language to write commands describing the contraction/manipulation. These commands are written to a DSL script which is created when the DSL backend is created. Therefore, for example, contracting a tensor network representing a quantum circuit with a DSL backend will not produce the output of the circuit immediately. To get the output, we must also call an execute_dsl_file function which will read the DSL commands in the DSL script and then perform the operations described there to produce the output.

While writing a DSL script for later execution introduces some additional complexity it brings with it a number of important advantages including:
- Separation between the computation and control which makes it easier to support new hardware platforms. For example to do the computation on a new hardware platform/gpu architecture only requires implementing a small subset of operations. In addition new network architectures, algorithms and contraction strategies can be added/updated without having to rewrite the low level implementation details (so longer as these are supported within DSL specification).
- The use of a DSL script also allows for preprocessing stages to apply optimisations in a seamless manner without affecting the layers above or below

Next we show an example of the tensor language that's generated by the DSL backend before we go through the specification for each of the commands.

## Tensor Language Example

To generate a tensor language script for the GHZ state preparation circuit shown in the introduction notebook, we only need to change the backend that's initialised to the DSL backend. To initialise a DSL backend and set it as the current backend used by PicoQuant we call the following constructor. The input arguments are the names of the DSL script, tensor date file and output date file respectively. (Note, if no output file is given, results will be written to the tensor data file by default.)

In [ ]:
using PicoQuant

# Initialise a DSL backend.
DSLBackend("dsl_file.tl", "tensor_file.h5", "output_file.h5")
tn = TensorNetworkCircuit(3)
add_input!(tn, "000")
add_gate!(tn, gate_tensor(:H), [1])
add_gate!(tn, gate_tensor(:CX), [2, 1])
add_gate!(tn, gate_tensor(:CX), [3, 2])
full_wavefunction_contraction!(tn, "vector")

This will write the DSL commands to a file called `dsl_file.tl` which looks like

In [ ]:
;cat dsl_file.tl

We can also examine the contents of the data file.

In [ ]:
using HDF5

h5open("tensor_file.h5") do io
    read(io)
end 

## The Domain Specific Language

The domain specific language consists of the following commands:
1. tensor - Command to load a tensor from the HDF5 file referenced by the DSL backend. Input arguments are: 
    1. A name for a variable to bind to the tensor data when it is loaded into memory 
    2. The name under which the tensor data is saved in the HDF5 file.
    
    
2. del - Command to delete a tensor. Input arguments are:
    1. The vaiable name of a tensor in memory to delete
    
    
3. save - Command to save a tensor to a given output file. Input arguments are:
    1. The tensor variable name, 
    2. The output HDF5 file name,
    3. The name to save the tensor under within the HDF5 file.


4. ncon - Command to contract two tensors $A$ and $B$ together and produce a new tensor $C$. Input arguments are 
    1. A name for the new tensor $C$, 
    2. The name for the first tensor $A$ 
    3. The ncon indices identifying which indices of $A$ to contract, 
    4. The name of the second tensor $B$,
    5. The ncon indices for the second tensor $B$.


5. reshape - Command to reshape a tensor. Input arguments are:
    1. The name of a tensor to reshape,
    2. An array of integers specifying the desired shape of the tensor.
    
    
6. decompose - Command to decompose a tensor using SVD. Input arguments are:
    1. The name of the tensor to decompose,
    2. An array of integers iddentifying the indices of the tensor to be considered the left indices during SVD.
    3. An array of integers iddentifying the right indices of the tensor during SVD.
    4. A real, positive number giving the threshold under which singular values will be discarded.


7. permute - Command to permute the axes of the given tensor. Input arguments are:
    1. The name of the tensor to permute the axes of
    2. A comma separated list of the axes to appear at each position

## Executing the DSL operations

After the commands have been written to the DSL script file and the tensor data written to the h5 file, these operations are ready to be executed. At present there is a reference implementation which executes these commands which is the same as that used for the InteractiveBackend. We plan to extend this in future to implement specific backends for different GPU architectures and clusters we plan to support.

To get the output of the circuit we created, we call the function execute_dsl_file and pass it the DSL script and the tensor data HDF5 file. This will save the output of the circuit under the name "result" in the output file given to the DSL backend when it was created.

In [ ]:
execute_dsl_file("dsl_file.tl", "tensor_file.h5")

We can read the result of the above contraction as follows:

In [ ]:
output = load_tensor_data(backend, :result)